In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python.ops.rnn_cell import LSTMCell, LSTMStateTuple
import helpers
import os
import datetime

#LSTMStateTuple(c,h)에서 c는 hidden state h는 output

tf.reset_default_graph()

In [2]:
lstm_size_encoder = 128
lstm_size_decoder = 256
vocab_size = 679 #띄어쓰기도 하나의 word야?
embed_size = 256# Document Vector로 Input된 총 문장수고 각 Vocab이 몇번 나왔는지 표기(?)
#max_length = 5 


#batch_size = 32
encoder_cell = LSTMCell(lstm_size_encoder)
decoder_cell = LSTMCell(lstm_size_decoder)
GO = 0
PAD = 0
EOS = 1

## Embeddings: Used int type embedding lookup

## Load Data

In [3]:
data_dir = "en_fr"
input_file_en = os.path.join(data_dir, "A_62_952_ADD1_en.snt")
lines_en = []
with open(input_file_en, "r", encoding="utf-8") as f_en:
    for line in f_en.readlines():
        line = line.replace('\n','')
        if not line == '':
            lines_en.append(line)
        
print(lines_en[-2])
#print(("Text loaded from '%s'") % (input_file))

A revised edition of the rules of procedure has been issued under the symbol A/520/Rev.17.


In [4]:
data_dir = "en_fr"
#save_dir = "eng_fre_dataset/multiUN.en/un/text/en-fr/2009"
input_file_fr = os.path.join(data_dir, "A_62_952_ADD1_fr.snt")
lines_fr = []
with open(input_file_fr, "r", encoding="utf-8") as f_fr:
    for line in f_fr.readlines():
        line = line.replace('\n','')
        if not line == '':
            lines_fr.append(line)
        
print(lines_fr[-2])
#print(("Text loaded from '%s'") % (input_file))

Une version révisée du Règlement intérieur a été publiée sous la cote A/520/Rev.17.


In [5]:
len(lines_en), len(lines_fr)

(67, 67)

In [6]:
word_data = []
for i in range(len(lines_en)):
    word_data.append((lines_en[i],lines_fr[i]))

In [7]:
word_data[:5]

[('Sixty-second session', 'Soixante-deuxième session'),
 ('Agenda item 121', "Point 121 de l'ordre du jour"),
 ('Revitalization of the work of the General Assembly',
  "Revitalisation des travaux de l'Assemblée générale"),
 ('Report of the Ad Hoc Working Group on the Revitalization of the General Assembly',
  "Rapport du Groupe de travail spécial sur la revitalisation de l'Assemblée générale"),
 ('Addendum', 'Additif')]

In [8]:
# word_data = [
#     ('bye world good', 'thank you word'),
#     ('sam gyup sal', 'ha ha'),
#     ('bye world hi hi hi', 'Goodyhi')
# ] * 100

In [9]:
x = [x for x, y in word_data]
y = [y for x, y in word_data]

In [10]:
x_list = []
for i in range(len(x)):
    x_text = x[i].split()
    x_list.append(x_text)
for i in range(1, len(x_list)):
    x_list[i] = set(x_list[i-1]).union(set(x_list[i]))
    x_words_set = x_list[len(x_list)-1]

In [11]:
y_list = []
for i in range(len(y)):
    y_text = y[i].split()
    y_list.append(y_text)
for i in range(1, len(y_list)):
    y_list[i] = set(y_list[i-1]).union(set(y_list[i]))
    y_words_set = y_list[len(y_list)-1]

In [12]:
word2numX = dict(zip(x_words_set, range(len(x_words_set))))
word2numY = dict(zip(y_words_set, range(len(y_words_set))))

In [13]:
num2wordX = dict(zip(word2numX.values(), word2numX.keys()))

In [14]:
# max_len 뽑음
numWordsList = []
for word in x:
    Words = word.split(' ')
    num = len(Words)
    numWordsList.append(num) 
max_len = max(numWordsList)

In [15]:
step1_X = [
    word.split() for word in x
]

In [16]:
second_list = []
for i in range(len(step1_X)):
    first_list = []
    for j in range(len(step1_X[i])):
        k = word2numX[step1_X[i][j]]
        first_list.append(k)
        padded_list = first_list
    second_list.append(padded_list)
x = second_list
x = np.array(x)

In [17]:
num2wordY = dict(zip(word2numY.values(), word2numY.keys()))

In [18]:
step1_Y = [
    word.split() for word in y
]

In [19]:
second_list = []
for i in range(len(step1_Y)):
    first_list = []
    for j in range(len(step1_Y[i])):
        k = word2numY[step1_Y[i][j]]
        first_list.append(k)
        padded_list = first_list
    second_list.append(padded_list)
y = second_list
y = np.array(y)

In [20]:
print(len(word2numX))
print(len(word2numY))


679
720


In [21]:
print(num2wordX)

{0: 'avoid', 1: 'indicate', 2: 'thereby', 3: 'Sixty-second', 4: 'obstacles', 5: 'revitalization).', 6: 'help', 7: 'States', 8: 'committees', 9: 'States,', 10: 'Nations', 11: 'has', 12: 'reorganized', 13: '46', 14: 'records', 15: 'Assembly.', 16: 'undertakes', 17: 'A/520/Rev.17.', 18: 'there', 19: 'carries', 20: 'to', 21: 'climate', 22: 'UNESCO', 23: 'opportunity', 24: 'languages,', 25: 'thematic', 26: 'provision', 27: 'Permanent', 28: "Department's", 29: 'photographers', 30: 'instruments', 31: 'plus', 32: 'Centre', 33: 'a', 34: 'indexes', 35: 'order', 36: 'all', 37: 'communications', 38: 'Management', 39: 'address', 40: 'areas', 41: 'would', 42: 'videoconferences', 43: 'Yearbook', 44: 'posted', 45: 'summaries', 46: 'mandates', 47: 'translation', 48: 'photographs', 49: 'may', 50: 'plenary,', 51: 'Rights', 52: 'Radio', 53: 'agenda,', 54: 'Conference,', 55: 'alerts', 56: 'necessary.', 57: 'group,', 58: 'implementation', 59: 'this', 60: 'etc.).', 61: 'list', 62: 'work', 63: 'representative

In [22]:
num2wordX[0]

'avoid'

In [23]:
len(x)

67

In [24]:
len(y)

67

## Embeddings

In [25]:
encoder_inputs = tf.placeholder(tf.int32, (None, None), "encoder_inputs")
encoder_inputs_length = tf.placeholder(tf.int32, (None, ), "encoder_inputs_length")
decoder_inputs = tf.placeholder(tf.int32, (None, None), "decoder_inputs")
decoder_targets = tf.placeholder(tf.int32, (None, None), "decoder_targets")

In [26]:
input_embedding = tf.Variable(tf.random_uniform((vocab_size, embed_size), -1.0, 1.0), name="input_embedding")
output_embedding = tf.Variable(tf.random_uniform((len(word2numY),embed_size), -1.0, 1.0), name="output_embedding")
embedded_input = tf.nn.embedding_lookup(input_embedding, encoder_inputs)
embedded_output = tf.nn.embedding_lookup(output_embedding, decoder_inputs)

## Encoder: Bidirectional LSTM

In [27]:
((enc_fw_outputs, enc_bw_outputs),
(enc_fw_last_state, enc_bw_last_state)) = (tf.nn.bidirectional_dynamic_rnn
                                            (cell_fw=encoder_cell,
                                             cell_bw=encoder_cell,
                                             inputs=embedded_input,
                                             sequence_length=encoder_inputs_length,
                                             dtype=tf.float32, time_major=True))

In [28]:
encoder_outputs = tf.concat((enc_fw_outputs, enc_bw_outputs), 2)
enc_last_state_c = tf.concat((enc_fw_last_state.c, enc_bw_last_state.c),1)
enc_last_state_h = tf.concat((enc_fw_last_state.h, enc_bw_last_state.h),1)
encoder_last_state = LSTMStateTuple(
    c=enc_last_state_c,
    h=enc_last_state_h
)

## Decoder: Attention

In [29]:
decoder_cell = LSTMCell(lstm_size_decoder)

In [30]:
encoder_max_time, batch_size = tf.unstack(tf.shape(encoder_inputs))

In [31]:
decoder_lengths = encoder_inputs_length + 22

In [32]:
#manually specifying since we are going to implement attention details for the decoder in a sec
#weights
W = tf.Variable(tf.random_uniform([lstm_size_decoder, vocab_size], -1, 1), dtype=tf.float32)
#bias
b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [33]:
assert EOS == 1 and PAD == 0

eos_time_slice = tf.ones([batch_size], dtype=tf.int32, name='EOS')
pad_time_slice = tf.zeros([batch_size], dtype=tf.int32, name='PAD')

#retrieves rows of the params tensor. The behavior is similar to using indexing with arrays in numpy
eos_step_embedded = tf.nn.embedding_lookup(input_embedding, eos_time_slice)
pad_step_embedded = tf.nn.embedding_lookup(input_embedding, pad_time_slice)

In [34]:
def loop_fn_initial():
    initial_elements_finished = (0 >= decoder_lengths)  # all False at the initial step
    #end of sentence
    initial_input = eos_step_embedded
    #last time steps cell state
    initial_cell_state = encoder_last_state
    #none
    initial_cell_output = None
    #none
    initial_loop_state = GO  # we don't need to pass any additional information
    return (initial_elements_finished,
            initial_input,
            initial_cell_state,
            initial_cell_output,
            initial_loop_state)

In [35]:
#attention mechanism --choose which previously generated token to pass as input in the next timestep
def loop_fn_transition(time, previous_output, previous_state, previous_loop_state):

    
    def get_next_input():
        #dot product between previous ouput and weights, then + biases
        output_logits = tf.add(tf.matmul(previous_output, W), b)
        #Logits simply means that the function operates on the unscaled output of 
        #earlier layers and that the relative scale to understand the units is linear. 
        #It means, in particular, the sum of the inputs may not equal 1, that the values are not probabilities 
        #(you might have an input of 5).
        #prediction value at current time step
        
        #Returns the index with the largest value across axes of a tensor.
        prediction = tf.argmax(output_logits, axis=1)
        #embed prediction for the next input
        next_input = tf.nn.embedding_lookup(input_embedding, prediction)
        return next_input
    
    
    elements_finished = (time >= decoder_lengths) # this operation produces boolean tensor of [batch_size]
                                                  # defining if corresponding sequence has ended

    
    
    #Computes the "logical and" of elements across dimensions of a tensor.
    finished = tf.reduce_all(elements_finished) # -> boolean scalar
    #Return either fn1() or fn2() based on the boolean predicate pred.
    input = tf.cond(finished, lambda: pad_step_embedded, get_next_input)
    
    #set previous to current
    state = previous_state
    output = previous_output + embedded_output[-1]
    loop_state = None

    return (elements_finished, 
            input,
            state,
            output,
            loop_state)

In [36]:
def loop_fn(time, previous_output, previous_state, previous_loop_state):
    if previous_state is None:    # time == 0
        assert previous_output is None and previous_state is None
        return loop_fn_initial()
    else:
        return loop_fn_transition(time, previous_output, previous_state, previous_loop_state)

#Creates an RNN specified by RNNCell cell and loop function loop_fn.
#This function is a more primitive version of dynamic_rnn that provides more direct access to the 
#inputs each iteration. It also provides more control over when to start and finish reading the sequence, 
#and what to emit for the output.
#ta = tensor array
decoder_outputs_ta, decoder_last_state, decoder_loop_state = tf.nn.raw_rnn(decoder_cell, loop_fn)
decoder_outputs = decoder_outputs_ta.stack()


In [37]:
decoder_outputs

<tf.Tensor 'TensorArrayStack/TensorArrayGatherV3:0' shape=(?, ?, 256) dtype=float32>

In [38]:
#to convert output to human readable prediction
#we will reshape output tensor

#Unpacks the given dimension of a rank-R tensor into rank-(R-1) tensors.
#reduces dimensionality
decoder_max_steps, decoder_batch_size, decoder_dim = tf.unstack(tf.shape(decoder_outputs))
#flettened output tensor
decoder_outputs_flat = tf.reshape(decoder_outputs, (-1, decoder_dim))
#pass flattened tensor through decoder
decoder_logits_flat = tf.add(tf.matmul(decoder_outputs_flat, W), b)

#prediction vals
decoder_logits = tf.reshape(decoder_logits_flat, (decoder_max_steps, decoder_batch_size, vocab_size))

In [39]:
#final prediction
decoder_prediction = tf.argmax(decoder_logits, 2)

## Optimizer

In [40]:
#cross entropy loss
#one hot encode the target values so we don't rank just differentiate
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(decoder_targets, depth=vocab_size, dtype=tf.float32),
    logits=decoder_logits,
)

#loss function
loss = tf.reduce_mean(stepwise_cross_entropy)
#train it 
train_op = tf.train.AdamOptimizer().minimize(loss)

In [41]:
type(x[0])

list

In [42]:
def next_feed():
    batch = x[:300]
    batch_y = y[:300]
    encoder_inputs_, encoder_input_lengths_ = helpers.batch(batch)
    decoder_inputs_, _ = helpers.batch(
        [[GO] + (sequence) + [EOS] for sequence in batch_y]
    )
    decoder_targets_, _ = helpers.batch(
        [(sequence) + [EOS] + [PAD] for sequence in batch_y]
    )
    
    return {
        encoder_inputs: encoder_inputs_,
        encoder_inputs_length: encoder_input_lengths_,
        decoder_inputs: decoder_inputs_,
        decoder_targets: decoder_targets_,
    }

In [43]:
def next_feed_test():
    batch_x_test = x[400:]
    dec_input = np.zeros((len(batch_x_test), 1)) + 0
    batch_y_test = y[400:]
    encoder_inputs_, encoder_input_lengths_ = helpers.batch(batch_x_test)
#     decoder_inputs_, _ = helpers.batch(
#         [[GO] + (sequence) + [EOS] + [PAD] * 3 for sequence in batch_y_test]
#     )
#     decoder_targets_, _ = helpers.batch(
#         [(sequence) + [EOS] + [PAD] * 4 for sequence in batch_y]
#     )
    return {
        encoder_inputs: encoder_inputs_,
        encoder_inputs_length: encoder_input_lengths_,
        decoder_inputs: dec_input,
    }, batch_x_test, batch_y_test

In [44]:
loss_track = []

In [ ]:
save_every = 50
save_dir = 'data/'
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [ ]:
max_batches = 1001
batches_in_epoch = 50
num_batch_save = 50

for batch in range(max_batches):
    fd = next_feed()
   
    _, batch_loss, batch_logits = sess.run([train_op, loss, decoder_logits], fd)
    if batch == 0 or batch % batches_in_epoch == 0:
        Epoch = batch // batches_in_epoch
        print('=============================EPOCH   :{:>4} ============================='.format(Epoch))
        print('batch {}/{}'.format(batch, max_batches))
        print('#####=====batch loss  :  {:>6.3f}'.format(batch_loss))
        predict_ = sess.run(decoder_prediction, fd)
        
#      loss_track.append(batch_loss)
# #     print(batch_loss)

#     if batch == 0 or batch % batches_in_epoch == 0:
#         print('batch {}'.format(batch))
#         print('  minibatch loss: {}'.format(sess.run(loss, fd)))
        
#         save progress every 10000 iterations
        for i, (inp, pred) in enumerate(zip(fd[encoder_inputs].T, predict_.T)):
            print('  sample {}:'.format(i + 1))
            print('    input     > {}'.format(inp))
            print('    prediction     > {}'.format(pred))
     

            
#             result_en = []
#             for i in range(len(inp)):
#                 temp_en = num2wordX[inp[i]]
#                 result_en.append(temp_en)
#             #print(result_en)
            
#             result_fr =[]
#             for i in range(len(pred)):
#                 temp_fr = num2wordY[pred[i]]
#                 result_fr.append(temp_fr)
#             print('input: {} ========================>\nprediction: {}'.format(result_en,result_fr))
            if i >= 2:
                break
        accuracy = np.mean(batch_logits.argmax(axis=-1)==predict_)
        print('#####=====training accuracy  : {:>6.3f}'.format(accuracy))
    
#         print(batch_logits.argmax(axis=-1))
#         print(predict_)
#         print()


=============================EPOCH   :   0 =============================
batch 0/1001
#####=====batch loss  :   9.484
  sample 1:
    input     > [  3 590   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0]
    prediction     > [189 189 189 189 189 189 189 189 189 189 189 189 189 189 189 281 189 189
 189 189 189 189 189 189   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [ ]:
####==== Test Code

_, source_batch, target_batch = next_feed_test()

dec_input = np.zeros((len(source_batch), 1)) + 0
#print(dec_input)
for i in range(len(num2wordY)):
    fd, _, _ = next_feed_test()
    batch_logits = sess.run([decoder_logits], fd)
    R = np.array(batch_logits)
    print(R[:,-1])
    prediction = R.argmax(axis=-1)
    print(prediction)
    dec_input = np.hstack([dec_input, prediction[None,None]])
    #print(dec_input)
    #print(target_batch)
#     if batch == 0 or batch % batches_in_epoch == 0:
#         #print(batch_logits)
#         print('batch {}'.format(batch))
#         print(batch_loss)
#         predict_ = sess.run(decoder_prediction, fd)
        
# #      loss_track.append(batch_loss)
# # #     print(batch_loss)

# #     if batch == 0 or batch % batches_in_epoch == 0:
# #         print('batch {}'.format(batch))
# #         print('  minibatch loss: {}'.format(sess.run(loss, fd)))
        
# #         save progress every 10000 iterations
#         for i, (inp, pred) in enumerate(zip(fd[encoder_inputs].T, predict_.T)):
#             print('  sample {}:'.format(i + 1))
#             print('    input     > {}'.format(inp))
#             print('    prediction     > {}'.format(pred))
#             if i >= 2:
#                 break
accuracy = np.mean(dec_input == target_batch)
print(accuracy)

#         print(batch_logits.argmax(axis=-1))
#         print(predict_)
#         print()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(loss_track)
print('loss {:.4f} after {} examples (batch_size={})'.format(loss_track[-1], len(loss_track)*batch_size, batch_size))